# Check GPU Acceleration with Apple Silicon

In [11]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Imports & Constants

In [12]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
NUM_CLASSES = 4

# Paths

In [13]:
dataset = 'models/keypoint_classifier/keypoint.csv'
model_save_path = 'models/keypoint_classifier/keypoint_classifier.hdf5'
tflite_save_path = 'models/keypoint_classifier/keypoint_classifier.tflite'

# Load Datasets

In [14]:
X_data = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y_data = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, train_size=0.8, random_state=RANDOM_SEED)

# Build Model

In [15]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Input((21 * 2, )),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(20, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [16]:
model.summary() 

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_2 (Dropout)             │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 20)             │           860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │            44 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,114 (4.35 KB)

 Trainable params: 1,114 (4.35 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Model Checkpoint Callback
model_save_path = model_save_path.replace('.hdf5', '.keras')
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Early Stopping Callback
es_callback = tf.keras.callbacks.EarlyStopping(
    patience=20,
    verbose=1
)

In [18]:
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy']
)

# Train

In [19]:
model.fit(
  X_train,
  y_train,
  epochs=1000,
  batch_size=128,
  validation_data=(X_test, y_test),
  callbacks=[cp_callback, es_callback]
)
# Evaluate
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

Epoch 1/1000
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.2982 - loss: 1.6144
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.2982 - loss: 1.6135 - val_accuracy: 0.2766 - val_loss: 1.3718
Epoch 2/1000
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3018 - loss: 1.5066
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3020 - loss: 1.5053 - val_accuracy: 0.3288 - val_loss: 1.3165
Epoch 3/1000
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3316 - loss: 1.4198
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.3317 - loss: 1.4191 - val_accuracy: 0.3612 - val_loss: 1.2528
Epoch 4/1000
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3414 - loss: 1.3537
Epoch 4: saving model to model/keypoint_classifier/keypoint_classifier.ke

# Save Model

In [20]:
model = tf.keras.models.load_model(model_save_path)
model.save(model_save_path, include_optimizer=False)

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open(tflite_save_path, 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/xf/xbw_8nmn2t1d0kklry2h0x6c0000gn/T/tmp8vls02o3/assets


INFO:tensorflow:Assets written to: /var/folders/xf/xbw_8nmn2t1d0kklry2h0x6c0000gn/T/tmp8vls02o3/assets


Saved artifact at '/var/folders/xf/xbw_8nmn2t1d0kklry2h0x6c0000gn/T/tmp8vls02o3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 42), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  13151986960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13151988880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13151981776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13151988112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13151988688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13151989840: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1742491598.802338 1703666 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742491598.802533 1703666 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1742491598.808312 1703666 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled


# Test at Inference

In [21]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()
# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]], dtype=np.float32))

interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

print(tflite_results)
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[[3.17269296e-05 1.95088476e-01 1.09247296e-04 8.04770470e-01]]
[3.17269296e-05 1.95088476e-01 1.09247296e-04 8.04770470e-01]
3


/Users/johnzhang/Desktop/Development/Projects/gesture-navigator/venv/lib/python3.11/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
